In [3]:
import pandas as pd
import numpy as np
import datetime as datetime
import os
os.getcwd()
dir = "/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data/"
os.chdir(dir)
os.getcwd()

'/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data'

In [58]:
keys = ['8to9', '9to10', '10to11', '11to12','12to13','13to14','14to15','15to16','16to17','17to18','18to19','19to20']

ec_accptresponse = ['15 to 30 minutes', '5 to 15 minutes', 'Less than 5 minutes']
eventcontingent_dictionary = {'Less than 5 minutes': 2.5, 
                              '15 to 30 minutes': 17.5, 
                              '5 to 15 minutes': 10
                             } 

In [5]:
# read data
contingent_ema = pd.read_csv(dir + 'eventcontingent-ema.csv')
eod_ema = pd.read_csv(dir + 'eod-ema.csv')

contingent_ema_alternative = pd.read_csv(dir + 'eventcontingent-ema-alternative.csv')
eod_ema_alternative = pd.read_csv(dir + 'eod-ema-alternative.csv')

contingent_ema_backup = pd.read_csv(dir + 'eventcontingent-ema-backup.csv')
eod_ema_backup = pd.read_csv(dir + 'eod-ema-backup.csv')

In [64]:
# Make a list of all contingent event-times between 8AM and 8PM
# Throw away observations for 'when_smoke' is nan or 
# 'More than 30 minutes' to ensure we can calculate a meaningful 
# quantity.
days_smoked = {}
for index, row in contingent_ema.iterrows():
    count += 1
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
        date = (time.year, time.month, time.day, time.hour)
    if row['participant_id'] not in days_smoked:
        days_smoked[row['participant_id']] = set()
    if 8 <= date[3] < 20 and row['when_smoke'] in ec_accptresponse:        
        days_smoked[row['participant_id']].add(date)

days_smoked

{201: {(2017, 6, 23, 18), (2017, 6, 24, 15), (2017, 6, 24, 19)},
 202: {(2017, 6, 26, 13),
  (2017, 6, 27, 10),
  (2017, 6, 27, 12),
  (2017, 7, 6, 15)},
 205: {(2017, 8, 18, 16),
  (2017, 8, 18, 19),
  (2017, 8, 19, 10),
  (2017, 8, 19, 15),
  (2017, 8, 20, 11),
  (2017, 8, 20, 16)},
 208: {(2017, 9, 24, 19),
  (2017, 9, 26, 18),
  (2017, 9, 27, 16),
  (2017, 9, 28, 9),
  (2017, 9, 28, 14)},
 209: {(2017, 9, 29, 14),
  (2017, 9, 29, 16),
  (2017, 9, 29, 19),
  (2017, 9, 30, 12)},
 211: {(2017, 10, 2, 16),
  (2017, 10, 2, 19),
  (2017, 10, 3, 11),
  (2017, 10, 3, 19),
  (2017, 10, 4, 10),
  (2017, 10, 4, 19),
  (2017, 10, 7, 12)},
 212: {(2017, 10, 13, 15), (2017, 10, 15, 10)},
 213: {(2017, 10, 16, 16),
  (2017, 10, 16, 19),
  (2017, 10, 17, 15),
  (2017, 10, 17, 18),
  (2017, 10, 18, 13),
  (2017, 10, 18, 15),
  (2017, 10, 18, 18)},
 214: {(2017, 10, 23, 13),
  (2017, 10, 23, 17),
  (2017, 10, 24, 9),
  (2017, 10, 24, 13),
  (2017, 10, 24, 18),
  (2017, 10, 25, 14),
  (2017, 10, 25, 

In [23]:
count = 0 
missing = 0
missing_days = 0
lst = []
events = set()
    
# store contingent events
for index, row in contingent_ema.iterrows():
    count += 1
    try:
        time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
    except:
        time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
        date = (time.year, time.month, time.day, time.hour)
    if row['participant_id'] not in days_smoked:
        continue
    if date not in days_smoked[row['participant_id']]:
        missing += 1
        if date not in events:
            missing_days += 1
            events.add(date)
        lst.append(index)

missing_in_eod = contingent_ema.iloc[lst]

In [18]:
def contingent_vs_eod(contingent_ema, eod_ema, alt_date=False):
    '''
    checks how reliable eod_ema is in tracking smoking events recorded in contingent_ema
    '''
    
    days_smoked = {}
    # store events in eod_ema
    for index, row in eod_ema.iterrows():
        if row['status'] == "MISSED":
            continue
        for i in keys:
            if row[i] == 1:
                try:
                    time = datetime.datetime.strptime(row['date'], '%m/%d/%Y %H:%M')
                except:
                    time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
                date = (time.year, time.month, time.day)
                if time.hour  == 0 or time.hour == 1:
                    date = (time.year, time.month, time.day-1)
                if row['participant_id'] not in days_smoked:
                    days_smoked[row['participant_id']] = set()
                days_smoked[row['participant_id']].add(date)
                break

    count = 0 
    missing = 0
    missing_days = 0
    lst = []
    events = set()
    
    # store contingent events
    for index, row in contingent_ema.iterrows():
        count += 1
        try:
            time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
        except:
            time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
        date = (time.year, time.month, time.day)
        if row['participant_id'] not in days_smoked:
            continue
        if date not in days_smoked[row['participant_id']]:
            missing += 1
            if date not in events:
                missing_days += 1
                events.add(date)
            lst.append(index)

    missing_in_eod = contingent_ema.iloc[lst]
    # percentage of contingent events missed by eod_ema
    print("Contingent vs. EOD EMA inconsistency percentage by entries is: ", missing/float(count))
    print("Contingent vs. EOD EMA inconsistency percentage by day is: ", missing_days/float(count))
    return missing_in_eod

In [16]:
contingent_vs_eod(contingent_ema, eod_ema)

('Contingent vs. EOD EMA inconsistency percentage by entries is: ', 0.15555555555555556)
('Contingent vs. EOD EMA inconsistency percentage by day is: ', 0.1)


,status,smoke,when_smoke,urge,cheerful,happy,angry,stress,sad,see_or_smell,access,smoking_location,participant_id,timestamp,offset,date,hour,minute,day_of_week
24,COMPLETED,Yes,Less than 5 minutes,5.0,6.0,6.0,1.0,1.0,1.0,3.0,4.0,3.0,208,1506605347172,-360,2017-09-28 09:29:07,9,29,Thursday
25,COMPLETED,Yes,Less than 5 minutes,4.0,5.0,5.0,1.0,1.0,1.0,4.0,4.0,4.0,208,1506623357682,-360,2017-09-28 14:29:17,14,29,Thursday
42,COMPLETED,Yes,Less than 5 minutes,3.0,4.0,4.0,2.0,1.0,2.0,3.0,4.0,4.0,213,1508268649534,-360,2017-10-17 15:30:49,15,30,Tuesday
43,COMPLETED,Yes,Less than 5 minutes,3.0,3.0,4.0,2.0,3.0,3.0,3.0,4.0,3.0,213,1508278495840,-360,2017-10-17 18:14:55,18,14,Tuesday
64,COMPLETED,Yes,Less than 5 minutes,3.0,2.0,3.0,3.0,2.0,3.0,3.0,4.0,4.0,216,1509139157669,-360,2017-10-27 17:19:17,17,19,Friday
66,COMPLETED,Yes,15 to 30 minutes,1.0,4.0,4.0,3.0,3.0,3.0,4.0,4.0,3.0,217,1509463135411,-360,2017-10-31 11:18:55,11,18,Tuesday
67,COMPLETED,Yes,5 to 15 minutes,6.0,4.0,4.0,2.0,2.0,2.0,3.0,3.0,3.0,217,1509483292098,-360,2017-10-31 16:54:52,16,54,Tuesday
68,COMPLETED,Yes,5 to 15 minutes,6.0,6.0,5.0,2.0,2.0,2.0,4.0,4.0,4.0,217,1509491756321,-360,2017-10-31 19:15:56,19,15,Tuesday
75,COMPLETED,Yes,15 to 30 minutes,5.0,4.0,4.0,3.0,4.0,3.0,3.0,4.0,4.0,218,1509809338129,-360,2017-11-04 11:28:58,11,28,Saturday
104,COMPLETED,Yes,Less than 5 minutes,6.0,4.0,4.0,1.0,1.0,2.0,3.0,4.0,4.0,220,1511883734335,-360,2017-11-28 10:42:14,10,42,Tuesday


In [15]:
contingent_vs_eod(contingent_ema_alternative, eod_ema_alternative)

('Contingent vs. EOD EMA inconsistency percentage by entries is: ', 0.13114754098360656)
('Contingent vs. EOD EMA inconsistency percentage by day is: ', 0.11475409836065574)


,status,smoke,when_smoke,urge,cheerful,happy,angry,stress,sad,see_or_smell,access,smoking_location,participant_id,timestamp,date,hour,minute,day_of_week
5,COMPLETED,Yes,Less than 5 minutes,4.0,2.0,2.0,4.0,4.0,1.0,4.0,4.0,4.0,224,1516648017420,2018-01-22 14:06:57,14,6,Monday
6,COMPLETED,Yes,Less than 5 minutes,3.0,3.0,4.0,2.0,2.0,1.0,4.0,4.0,4.0,224,1516728422997,2018-01-23 12:27:02,12,27,Tuesday
10,COMPLETED,Yes,Less than 5 minutes,3.0,6.0,6.0,1.0,1.0,1.0,3.0,4.0,4.0,226,1518221786577,2018-02-09 19:16:26,19,16,Friday
17,COMPLETED,Yes,Less than 5 minutes,5.0,5.0,4.0,4.0,3.0,2.0,4.0,3.0,4.0,226,1518625473088,2018-02-14 11:24:33,11,24,Wednesday
18,COMPLETED,Yes,5 to 15 minutes,5.0,2.0,3.0,4.0,2.0,2.0,4.0,4.0,4.0,226,1518639385747,2018-02-14 15:16:25,15,16,Wednesday
19,COMPLETED,Yes,Less than 5 minutes,6.0,6.0,6.0,1.0,1.0,1.0,4.0,4.0,4.0,226,1518794236697,2018-02-16 10:17:16,10,17,Friday
21,MISSED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226,1519221279126,2018-02-21 08:54:39,8,54,Wednesday
60,COMPLETED,Yes,15 to 30 minutes,6.0,6.0,6.0,6.0,6.0,6.0,3.0,3.0,3.0,227,1519394953277,2018-02-23 09:09:13,9,9,Friday


In [17]:
contingent_vs_eod(contingent_ema_backup, eod_ema_backup)

('Contingent vs. EOD EMA inconsistency percentage by entries is: ', 0.11351351351351352)
('Contingent vs. EOD EMA inconsistency percentage by day is: ', 0.07567567567567568)


,status,smoke,when_smoke,urge,cheerful,happy,angry,stress,sad,see_or_smell,access,smoking_location,participant_id,timestamp,date,hour,minute,day_of_week
41,COMPLETED,Yes,Less than 5 minutes,5,6,6,1,1,1,3,4,3,208,1.506610e+12,9/28/17 9:29,9,29,Thursday
42,COMPLETED,Yes,Less than 5 minutes,4,5,5,1,1,1,4,4,4,208,1.506620e+12,9/28/17 14:29,14,29,Thursday
81,COMPLETED,Yes,Less than 5 minutes,3,2,3,3,2,3,3,4,4,216,1.509140e+12,10/27/17 17:19,17,19,Friday
83,COMPLETED,Yes,15 to 30 minutes,1,4,4,3,3,3,4,4,3,217,1.509460e+12,10/31/17 11:18,11,18,Tuesday
84,COMPLETED,Yes,5 to 15 minutes,6,4,4,2,2,2,3,3,3,217,1.509480e+12,10/31/17 16:54,16,54,Tuesday
85,COMPLETED,Yes,5 to 15 minutes,6,6,5,2,2,2,4,4,4,217,1.509490e+12,10/31/17 19:15,19,15,Tuesday
92,COMPLETED,Yes,15 to 30 minutes,5,4,4,3,4,3,3,4,4,218,1.509810e+12,11/4/17 11:28,11,28,Saturday
122,COMPLETED,Yes,Less than 5 minutes,6,4,4,1,1,2,3,4,4,220,1.511880e+12,11/28/17 10:42,10,42,Tuesday
123,COMPLETED,Yes,Less than 5 minutes,5,3,3,4,4,4,4,4,4,220,1.511900e+12,11/28/17 14:01,14,1,Tuesday
124,COMPLETED,Yes,Less than 5 minutes,4,3,3,4,6,5,3,4,4,220,1.511910e+12,11/28/17 17:34,17,34,Tuesday
